# Merging and Preparing dataset for Clustering

At this poin all the indivudual EDA for each dataset has been completed, the next step would be merging all into one dataset.

### Postcode Data

In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
# POSTCODE DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_postcode_data.csv"
# Load the CSV file into a DataFrame
postcodeDf = pd.read_csv(file_path)
# Display the first few rows of the DataFrame to verify its content
postcodeDf.head()

,District,District Code,Lower layer super output area,Ward,Ward Code,Postcode,Postcode district,London zone,Latitude,Longitude,Nearest station,Distance to station,Average Income,Index of Multiple Deprivation,Rural/urban
0,Bromley,E09000006,Bromley 018B,Bromley Town,E05013991,BR1 1AA,BR1,5,51.401546,0.015415,Bromley South,0.218254,69600,24305,Urban major conurbation
1,Bromley,E09000006,Bromley 008B,Bromley Town,E05013991,BR1 1AB,BR1,4,51.406333,0.015208,Bromley North,0.253666,60000,13716,Urban major conurbation
2,Bromley,E09000006,Bromley 018C,Bromley Town,E05013991,BR1 1AE,BR1,4,51.404543,0.014195,Bromley North,0.462939,69600,20694,Urban major conurbation
3,Bromley,E09000006,Bromley 018B,Bromley Town,E05013991,BR1 1AF,BR1,5,51.401392,0.014948,Bromley South,0.227662,69600,24305,Urban major conurbation
4,Bromley,E09000006,Bromley 018B,Bromley Town,E05013991,BR1 1AG,BR1,5,51.401392,0.014948,Bromley South,0.227662,69600,24305,Urban major conurbation


### Crime Data

In [4]:
# CRIME DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_crime_data.csv"
# Load the CSV file into a DataFrame
crimeDf = pd.read_csv(file_path)
# Display the first few rows of the DataFrame to verify its content
crimeDf.head()

,LSOA Name,Ward,Arson and Criminal Damage,Burglary,Drug Offences,Miscellaneous Crimes Against Society,Possession of Weapons,Public Order Offences,Robbery,Theft,Vehicle Offences,Violence Against the Person
0,Barking and Dagenham 001A,Chadwell Heath,27,9,26,4,10,24,10,31,48,145
1,Barking and Dagenham 001B,Chadwell Heath,17,6,27,5,4,18,4,21,34,104
2,Barking and Dagenham 001C,Chadwell Heath,18,6,14,1,5,20,5,21,49,85
3,Barking and Dagenham 001D,Chadwell Heath,44,23,32,7,7,49,15,66,115,162
4,Barking and Dagenham 002A,Chadwell Heath,22,16,16,3,5,23,2,25,50,117


In [5]:
# List of columns to sum up for 'Total Crimes'
crime_columns = [
    "Arson and Criminal Damage",
    "Burglary",
    "Drug Offences",
    "Miscellaneous Crimes Against Society",
    "Possession of Weapons",
    "Public Order Offences",
    "Robbery",
    "Theft",
    "Vehicle Offences",
    "Violence Against the Person",
]

In [7]:
# Create a new DataFrame from 'crimeDf'
# Initialize it with all columns from 'crimeDf'
crimesDf = crimeDf.copy()
# Add 'Total Crimes' column as the sum of all specified crime columns
crimesDf["Total Crimes"] = crimesDf[crime_columns].sum(axis=1)
# Rename 'Ward' column to 'Wards'
crimesDf.rename(columns={"Ward": "Wards"}, inplace=True)
# Sort the DataFrame alphabetically by the 'Ward' column
crimesDf = crimesDf.sort_values(by="LSOA Name")
# Drop the individual crime columns from the new DataFrame
crimesDf.drop(crime_columns, axis=1, inplace=True)
# Display the first few rows of the new DataFrame to verify the changes
crimesDf.head()
crimesDf.shape


(4553, 3)

In [8]:
# Filter postcodeDf to include only rows with 'Lower layer super output area' matching 'LSOA Name' in crimeDf
filtered_postcodeDf = postcodeDf[
    postcodeDf["Lower layer super output area"].isin(crimeDf["LSOA Name"])
]
# Ensure all required columns are present
required_columns = [
    "District",
    "Lower layer super output area",
    "Ward",
    "Postcode",
    "Postcode district",
    "London zone",
    "Latitude",
    "Longitude",
    "Average Income",
    "Index of Multiple Deprivation",
    "Rural/urban",
]

# Check if all required columns are in the DataFrame, add missing columns as NaN
for col in required_columns:
    if col not in filtered_postcodeDf.columns:
        filtered_postcodeDf[col] = np.nan
# Reorder the DataFrame according to the specified column order
filtered_postcodeDf = filtered_postcodeDf[required_columns]
# Sort the DataFrame alphabetically by the 'Ward' column
filtered_postcodeDf_sorted = filtered_postcodeDf.sort_values(
    by="Lower layer super output area"
).reset_index(drop=True)
# Display the first few rows of the filtered and re-ordered DataFrame to verify the changes
filtered_postcodeDf_sorted.head()
filtered_postcodeDf_sorted.shape


# Perform the merge between the filtered postcode DataFrame and crimes DataFrame
merged_df = filtered_postcodeDf_sorted.merge(
    crimesDf[["LSOA Name", "Total Crimes"]],
    left_on="Lower layer super output area",
    right_on="LSOA Name",
    how="left",
)
# Drop unnecessary columns immediately after merging
merged_df.drop(["LSOA Name"], axis=1, inplace=True)
# Find the index of 'Average Income' to determine where to insert 'Total Crimes'
average_income_index = merged_df.columns.get_loc("Average Income")
# Insert 'Total Crimes' right after 'Average Income' without specifying all columns manually
# +1 because insert places the column before the specified index, so adding 1 places it right after
merged_df.insert(
    average_income_index + 1, "Total Crimes", merged_df.pop("Total Crimes")
)
# Display the first few rows to verify the changes
merged_df.head()
#merged_df.shape

,District,Lower layer super output area,Ward,Postcode,Postcode district,London zone,Latitude,Longitude,Average Income,Total Crimes,Index of Multiple Deprivation,Rural/urban
0,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5PU,RM6,5,51.587985,0.134785,43600,334,5097,Urban major conurbation
1,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5LJ,RM6,5,51.584118,0.137387,43600,334,5097,Urban major conurbation
2,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5QJ,RM6,5,51.588871,0.140933,43600,334,5097,Urban major conurbation
3,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5NB,RM6,5,51.586918,0.133147,43600,334,5097,Urban major conurbation
4,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5ND,RM6,5,51.587073,0.131523,43600,334,5097,Urban major conurbation


### Rent Data

In [9]:
# RENT PRICE DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_rent_data.csv"
# Load the CSV file into a DataFrame
rentDf = pd.read_csv(file_path)
# Display the first few rows of the DataFrame to verify its content
rentDf.head()
#rentDf.shape
#rentDf.columns

,Borough,Bedroom Category,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price
0,Barking and Dagenham,Room,560,500,567,720
1,Barking and Dagenham,Studio,809,700,750,913
2,Barking and Dagenham,One Bedroom,"1,124","1,000","1,100","1,227"
3,Barking and Dagenham,Two Bedrooms,"1,361","1,200","1,350","1,500"
4,Barking and Dagenham,Three Bedrooms,"1,573","1,425","1,550","1,700"


In [11]:
# Convert rent price columns to numeric, ensuring to replace commas and convert
rent_price_columns = [
    "Average Rent Price",
    "Budget-Friendly Rent Price",
    "Median",
    "Premium Rents Price",
]
for column in rent_price_columns:
    rentDf[column] = pd.to_numeric(rentDf[column].str.replace(",", ""), errors="coerce")

# Group by 'Borough' and calculate the mean for each price-related column
average_prices_per_borough = (
    rentDf.groupby("Borough")[rent_price_columns].mean().reset_index()
)

# Display the results
average_prices_per_borough.head()

,Borough,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price
0,Barking and Dagenham,1226.333333,1095.833333,1202.833333,1376.666667
1,Barnet,1657.666667,1408.833333,1617.666667,1816.666667
2,Bexley,1216.833333,1020.333333,1170.666667,1423.833333
3,Brent,1615.500000,1376.000000,1553.666667,1790.000000
4,Bromley,1437.000000,1240.833333,1377.166667,1558.333333


In [12]:
# Merge merged_df with average_prices_per_borough to combine related data
# 'District' in merged_df and 'Borough' in average_prices_per_borough are needed for accurate merging
merged_df = merged_df.merge(
    average_prices_per_borough, left_on="District", right_on="Borough", how="left"
)

# Remove the 'Borough' column post-merge as it duplicates the 'District' information
merged_df.drop("Borough", axis=1, inplace=True, errors="ignore")

# Display the first few rows of merged_df to confirm successful merge and column removal
merged_df.head()
#merged_df.shape

,District,Lower layer super output area,Ward,Postcode,Postcode district,London zone,Latitude,Longitude,Average Income,Total Crimes,Index of Multiple Deprivation,Rural/urban,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price
0,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5PU,RM6,5,51.587985,0.134785,43600,334,5097,Urban major conurbation,1226.333333,1095.833333,1202.833333,1376.666667
1,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5LJ,RM6,5,51.584118,0.137387,43600,334,5097,Urban major conurbation,1226.333333,1095.833333,1202.833333,1376.666667
2,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5QJ,RM6,5,51.588871,0.140933,43600,334,5097,Urban major conurbation,1226.333333,1095.833333,1202.833333,1376.666667
3,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5NB,RM6,5,51.586918,0.133147,43600,334,5097,Urban major conurbation,1226.333333,1095.833333,1202.833333,1376.666667
4,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5ND,RM6,5,51.587073,0.131523,43600,334,5097,Urban major conurbation,1226.333333,1095.833333,1202.833333,1376.666667


### House Data

In [13]:
# HOUSE PRICE DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_house_data.csv"
# Load the CSV file into a DataFrame
pd.set_option("display.float_format", lambda x: "%.3f" % x)
houseDf = pd.read_csv(file_path)
# Rename 'District' column to 'Districts'
houseDf.rename(columns={"District": "Districts"}, inplace=True)
# Display the first few rows of the DataFrame to verify its content
houseDf.head()

,Districts,Mean average price,Annual mean average flat price,Annual mean average terraced price,Annual mean average semi price,Annual mean average detached price
0,Barking and Dagenham,355457.750,359436.750,235446.500,393380.000,444454.250
1,Barnet,902226.167,859164.500,481503.917,829460.167,1030449.417
2,Bexley,422608.909,416798.182,266897.636,425175.909,502805.600
3,Brent,536503.000,558503.400,407619.400,648485.000,749914.400
4,Bromley,535593.727,588873.000,319125.182,511276.364,640550.273


In [14]:
# Merge merged_df with houseDf to combine related data
# 'District' in merged_df and 'Districts' in houseDf are needed for accurate merging
merged_df = merged_df.merge(
    houseDf, left_on="District", right_on="Districts", how="left"
)

# Remove the 'Districs' column post-merge as it duplicates the 'District' information
merged_df.drop("Districts", axis=1, inplace=True, errors="ignore")

# Display the first few rows of merged_df to confirm successful merge and column removal
merged_df.head()
#merged_df.shape

,District,Lower layer super output area,Ward,Postcode,Postcode district,London zone,Latitude,Longitude,Average Income,Total Crimes,Index of Multiple Deprivation,Rural/urban,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price,Mean average price,Annual mean average flat price,Annual mean average terraced price,Annual mean average semi price,Annual mean average detached price
0,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5PU,RM6,5,51.588,0.135,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250
1,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5LJ,RM6,5,51.584,0.137,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250
2,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5QJ,RM6,5,51.589,0.141,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250
3,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5NB,RM6,5,51.587,0.133,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250
4,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5ND,RM6,5,51.587,0.132,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250


### Businesses Data

In [15]:
# BUSINESSES DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_businesses_data.csv"
# Load the CSV file into a DataFrame
businessesDf = pd.read_csv(file_path)
# Rename 'District' column to 'Districts'
businessesDf.rename(columns={"District": "Districts"}, inplace=True)
# Display the first few rows of the DataFrame to verify its content
businessesDf.head()

,District Code,Districts,"Agriculture, Forestry & Fishing",Manufacturing & Production,Building & Construction,Automotive Services,Grocery stores & Wholesale,Shops & Retail,Transportation & Storage,Hotels & Restaurants,Tech & Communication,Finance & Insurance,Real Estate Services,"Professional, Scientific & Technical",Business Administration & Support Services,Public Sector & Defence,Education,Healthcar Services,"Arts, Entertainment, Recreation & Other Services",Total
0,E09000001,City of London,30,795,655,25,650,710,220,430,"2,610","3,405",990,"7,580","2,350",5,245,315,960,"21,975"
1,E09000002,Barking and Dagenham,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,"7,885"
2,E09000003,Barnet,45,710,"3,680",365,"1,310","2,500",655,970,"2,210",475,"2,425","4,510","2,150",5,505,"1,190","1,555","25,260"
3,E09000004,Bexley,10,395,"2,090",205,295,625,435,495,690,90,235,"1,160",725,5,185,365,535,"8,540"
4,E09000005,Brent,5,580,"2,455",435,885,"1,660",775,920,"1,350",175,840,"2,150","1,300",0,280,640,"1,055","15,505"


In [16]:
# Merge merged_df with businessesDf to combine related data
# 'District' in merged_df and 'Districts' in businessesDf are needed for accurate merging
merged_df = merged_df.merge(
    businessesDf, left_on="District", right_on="Districts", how="left"
)

# Drop the 'Districts, District Code, Total' column as it's no longer needed since i got in the mergedDf 'District'
merged_df.drop(
    ["Districts", "District Code", "Total"], axis=1, inplace=True, errors="ignore"
)

# Display the first few rows to verify the changes
merged_df.head()
#merged_df.shape

,District,Lower layer super output area,Ward,Postcode,Postcode district,London zone,Latitude,Longitude,Average Income,Total Crimes,Index of Multiple Deprivation,Rural/urban,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price,Mean average price,Annual mean average flat price,Annual mean average terraced price,Annual mean average semi price,Annual mean average detached price,"Agriculture, Forestry & Fishing",Manufacturing & Production,Building & Construction,Automotive Services,Grocery stores & Wholesale,Shops & Retail,Transportation & Storage,Hotels & Restaurants,Tech & Communication,Finance & Insurance,Real Estate Services,"Professional, Scientific & Technical",Business Administration & Support Services,Public Sector & Defence,Education,Healthcar Services,"Arts, Entertainment, Recreation & Other Services"
0,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5PU,RM6,5,51.588,0.135,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345
1,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5LJ,RM6,5,51.584,0.137,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345
2,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5QJ,RM6,5,51.589,0.141,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345
3,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5NB,RM6,5,51.587,0.133,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345
4,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5ND,RM6,5,51.587,0.132,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345


### Green Spaces Data

In [23]:
# GREEN SPACES DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_park_data.csv"
# Load the CSV file into a DataFrame
greenSpacesDf = pd.read_csv(file_path)
# Rename 'Latitude' column to 'Lat' and 'Longitude' column to 'Lon'
greenSpacesDf.rename(columns={"Latitude": "Lat"}, inplace=True)
greenSpacesDf.rename(columns={"Longitude": "Lon"}, inplace=True)
# Display the first few rows of the DataFrame to verify its content
greenSpacesDf.head()
#greenSpacesDf.shape

,SiteName,Lat,Lon,Closest Postcode,Postcode District,Total parks in Postcode,Total parks in Postcode District
0,Warley Street Drain,51.563,0.301,RM14 3PA,RM14,1,28
1,Thames Chase Community Forest East,51.551,0.288,RM14 3NS,RM14,2,28
2,St Mary Magdelane Churchyard,51.540,0.288,RM14 3QH,RM14,1,28
3,Thames Chase Community Forest,51.552,0.282,RM14 3NS,RM14,2,28
4,Belhus Woods Country Park,51.520,0.259,RM15 4XJ,RM15,1,1


In [24]:
# Create a dictionary for mapping 'Postcode district' to 'District'
district_mapping = postcodeDf.set_index("Postcode district")["District"].to_dict()

# Map the 'Postcode District' in greenSpacesDf to get the corresponding 'Districts'
greenSpacesDf["Districts"] = greenSpacesDf["Postcode District"].map(district_mapping)

# Calculate the total number of parks per 'Postcode District'
greenSpacesDf["Total parks in District"] = greenSpacesDf.groupby("Districts")[
    "SiteName"
].transform("count")

# Verify the changes
greenSpacesDf.head()
greenSpacesDf.shape

(4350, 9)

In [25]:
# Drop the specified columns
greenSpacesDf.drop(
    columns=[
        "SiteName",
        "Lat",
        "Lon",
        "Closest Postcode",
        "Postcode District",
        "Total parks in Postcode",
        "Total parks in Postcode District",
    ],
    inplace=True,
)

# Drop duplicates based on the "Districts" column
greenSpacesDf = greenSpacesDf.drop_duplicates(subset=["Districts"])

# Reset the index
greenSpacesDf.reset_index(drop=True, inplace=True)

# Display the DataFrame to verify changes
greenSpacesDf
print("Shape of the DataFrame:", greenSpacesDf.shape)


Shape of the DataFrame: (33, 2)


In [26]:
# Merge merged_df with greenSpacesDf to combine related data
# 'District' in merged_df and 'Districts' in greenSpacesDf are needed for accurate merging
merged_df = merged_df.merge(
    greenSpacesDf, left_on="District", right_on="Districts", how="left"
)

# Drop the 'Districts' column as it's no longer needed since i got in the mergedDf 'District'
merged_df.drop(
    ["Districts"],
    axis=1,
    inplace=True,
    errors="ignore",
)

# Display the first few rows to verify the changes
merged_df.head()
#merged_df.shape

,District,Lower layer super output area,Ward,Postcode,Postcode district,London zone,Latitude,Longitude,Average Income,Total Crimes,Index of Multiple Deprivation,Rural/urban,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price,Mean average price,Annual mean average flat price,Annual mean average terraced price,Annual mean average semi price,Annual mean average detached price,"Agriculture, Forestry & Fishing",Manufacturing & Production,Building & Construction,Automotive Services,Grocery stores & Wholesale,Shops & Retail,Transportation & Storage,Hotels & Restaurants,Tech & Communication,Finance & Insurance,Real Estate Services,"Professional, Scientific & Technical",Business Administration & Support Services,Public Sector & Defence,Education,Healthcar Services,"Arts, Entertainment, Recreation & Other Services",Total parks in District
0,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5PU,RM6,5,51.588,0.135,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73
1,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5LJ,RM6,5,51.584,0.137,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73
2,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5QJ,RM6,5,51.589,0.141,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73
3,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5NB,RM6,5,51.587,0.133,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73
4,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5ND,RM6,5,51.587,0.132,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73


### Religion Data

In [28]:
# RELIGION DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_religion_data.csv"
# Load the CSV file into a DataFrame
religionDf = pd.read_csv(file_path)
# Display the first few rows of the DataFrame to verify its content
religionDf.head()

,LSOA Name,Ward,Christian,Buddhist,Hindu,Jewish,Muslim (Islam),Sikh,Other religion,No religion,Religion not stated
0,Camden 001A,Highgate,562.000,21,10.000,18.000,121.000,1,8,468.000,221
1,Camden 001B,Highgate,615.000,21,8.000,43.000,66.000,0,25,435.000,368
2,Camden 001C,Highgate,608.000,15,6.000,81.000,36.000,3,16,551.000,298
3,Camden 001D,Highgate,598.000,12,7.000,21.000,159.000,0,14,501.000,277
4,Camden 001E,Highgate,634.000,5,9.000,206.000,16.000,0,6,470.000,364


In [29]:
# Create a subframe for testing
df_sub = religionDf.copy()

# Specify the religion columns to consider
religions_columns = [
    "Christian",
    "Buddhist",
    "Hindu",
    "Jewish",
    "Muslim (Islam)",
    "Sikh",
    "Other religion",
    "No religion",
    "Religion not stated",
]
religionDf.shape

(4835, 11)

In [30]:
# Function to return n-th largest categories
def nth_largest_categories(row, n):
    sorted_row = row.sort_values(ascending=False)
    if n <= len(sorted_row):
        return sorted_row.index[n - 1]
    else:
        return np.nan


# Apply function to find the majority first, second, and third most common religions
df_sub["Major Religion"] = df_sub[religions_columns].apply(
    lambda row: nth_largest_categories(row, 1), axis=1
)
df_sub["Second Major Religion"] = df_sub[religions_columns].apply(
    lambda row: nth_largest_categories(row, 2), axis=1
)
df_sub["Third Major Religion"] = df_sub[religions_columns].apply(
    lambda row: nth_largest_categories(row, 3), axis=1
)
# Display the first few rows to verify the 'Majority Religion' column
df_sub[["LSOA Name", "Major Religion", "Second Major Religion", "Third Major Religion"]]


# Perform one-hot encoding for 'Majority Religion', 'Second Religion', 'Third Religion'
majority_onehot = pd.get_dummies(df_sub["Major Religion"], prefix="Majority")
second_onehot = pd.get_dummies(df_sub["Second Major Religion"], prefix="Second")
third_onehot = pd.get_dummies(df_sub["Third Major Religion"], prefix="Third")

# Concatenate the original DataFrame with the new one-hot encoded DataFrames
religionDf_encoded = pd.concat(
    [df_sub, majority_onehot, second_onehot, third_onehot], axis=1
)
# Drop the original categorical columns after encoding (attempting clustering without one-hot encoding can be an option to try)
religionDf_encoded.drop(
    [
        "Christian",
        "Buddhist",
        "Hindu",
        "Jewish",
        "Muslim (Islam)",
        "Sikh",
        "Other religion",
        "No religion",
        "Religion not stated",
        "Major Religion",
        "Second Major Religion",
        "Third Major Religion",
    ],
    axis=1,
    inplace=True,
)

# Rename 'Ward' column to 'Wards'
religionDf_encoded.rename(columns={"Ward": "Wards"}, inplace=True)
# Display the first few rows to verify the changes
religionDf_encoded.head()


,LSOA Name,Wards,Majority_Christian,Majority_Hindu,Majority_Jewish,Majority_Muslim (Islam),Majority_No religion,Majority_Religion not stated,Majority_Sikh,Second_Christian,Second_Hindu,Second_Jewish,Second_Muslim (Islam),Second_No religion,Second_Religion not stated,Second_Sikh,Third_Christian,Third_Hindu,Third_Jewish,Third_Muslim (Islam),Third_No religion,Third_Other religion,Third_Religion not stated,Third_Sikh
0,Camden 001A,Highgate,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
1,Camden 001B,Highgate,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
2,Camden 001C,Highgate,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
3,Camden 001D,Highgate,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
4,Camden 001E,Highgate,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False


In [31]:
# Merge merged_df with religionDf_encoded to combine related data
# 'Lower layer super output area' in merged_df and 'LSOA Name' in religionDf_encoded are needed for accurate merging
merged_df = merged_df.merge(
    religionDf_encoded,
    left_on="Lower layer super output area",
    right_on="LSOA Name",
    how="left",
)

# Drop 'LSOA Name' and 'Wards' from the merged DataFrame
merged_df.drop(["LSOA Name", "Wards"], axis=1, inplace=True, errors="ignore")
# Display the first few rows to verify the changes
merged_df.head()
#merged_df.shape

,District,Lower layer super output area,Ward,Postcode,Postcode district,London zone,Latitude,Longitude,Average Income,Total Crimes,Index of Multiple Deprivation,Rural/urban,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price,Mean average price,Annual mean average flat price,Annual mean average terraced price,Annual mean average semi price,Annual mean average detached price,"Agriculture, Forestry & Fishing",Manufacturing & Production,Building & Construction,Automotive Services,Grocery stores & Wholesale,Shops & Retail,Transportation & Storage,Hotels & Restaurants,Tech & Communication,Finance & Insurance,Real Estate Services,"Professional, Scientific & Technical",Business Administration & Support Services,Public Sector & Defence,Education,Healthcar Services,"Arts, Entertainment, Recreation & Other Services",Total parks in District,Majority_Christian,Majority_Hindu,Majority_Jewish,Majority_Muslim (Islam),Majority_No religion,Majority_Religion not stated,Majority_Sikh,Second_Christian,Second_Hindu,Second_Jewish,Second_Muslim (Islam),Second_No religion,Second_Religion not stated,Second_Sikh,Third_Christian,Third_Hindu,Third_Jewish,Third_Muslim (Islam),Third_No religion,Third_Other religion,Third_Religion not stated,Third_Sikh
0,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5PU,RM6,5,51.588,0.135,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
1,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5LJ,RM6,5,51.584,0.137,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
2,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5QJ,RM6,5,51.589,0.141,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
3,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5NB,RM6,5,51.587,0.133,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
4,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5ND,RM6,5,51.587,0.132,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False


### Religion Data

In [32]:
# ETHNICITY DATA
# Define the path to the CSV file
file_path = "/Users/alessandrosuarez_/Desktop/lastYRofUniShii/IndividualProjectIN3007/Deliverables/Workspace/data/processed/cleaned_ethniciy_data.csv"
# Load the CSV file into a DataFrame
ethnicityDf = pd.read_csv(file_path)
# Display the first few rows of the DataFrame to verify its content
ethnicityDf.head()
#ethnicityDf.shape
#ethnicityDf.columns

,LSOA Name,Ward,Afghan,African,African/Arab,Albanian,Anglo Indian,Any other ethnic group,Arab,Argentinian,Australian/New Zealander,Baltic States,"Bangladeshi, British Bangladeshi",Black British,Black European,Black and Asian,Black and Chinese,Black and White,Black/African American,Bosnian,Brazilian,British Asian,Burmese,Cambodia,Caribbean,Caribbean Asian,Chilean,Chinese,Chinese and White,Colombian,Commonwealth of (Russian) Independent States,Croatian,Cuban,Cypriot (part not stated),East African Asian,Ecuadorian,English/Welsh/Scottish/Northern Irish/British,European Mixed,Filipino,Greek,Greek Cypriot,Gypsy or Irish Traveller,Indian or British Indian,Indonesian,Iranian,Irish,Israeli,Italian,Japanese,Kashmiri,Korean,Kosovan,Kurdish,Latin/South/Central American,Malaysian,Mexican,Moroccan,Multi-ethnic islands,Nepalese (includes Gurkha),Nigerian,North African,North American,Other Eastern European,Other Middle East,Other Western European,Pakistani or British Pakistani,Peruvian,Polish,Polynesia/Micronesia/Melanesia,Punjabi,Serbian,Sinhalese,Somali,Somalilander,South Asian and Chinese,Sri Lankan,Taiwanese,Tamil,Thai,Turkish,Turkish Cypriot,Venezuelan,Vietnamese,White African,White Caribbean,White and Arab,White and Asian,White and Black African,White and Black Caribbean,White and East Asian,White and North African or Middle Eastern,White and South Asian
0,Camden 001A,Highgate,40,662.000,0,28,0,176,36,8,64,0,216.000,452.000,0,0,4,0,0,8,4,0,0,0,862.000,0,0,78,0,4,16,4,0,12,0,0,3136.000,84,12,90,12,0,52.000,0,56,11296.000,0,28,12,0,24,24,28,44,0,0,8,4,0,0,0,56,28,4,128,12,0,28,0,0,0,0,32,0,0,4,0,0,16,58,4,0,0,4,12,0,108,80,152,0,0,0
1,Camden 001B,Highgate,0,658.000,0,4,0,368,16,0,36,0,104.000,512.000,0,0,0,0,0,0,12,0,0,0,888.000,0,0,40,0,4,32,0,0,12,0,0,3424.000,200,68,110,20,4,36.000,0,24,12298.000,0,24,40,0,12,0,4,88,0,0,8,16,4,0,18,76,28,16,196,8,0,32,0,0,0,0,50,20,0,36,0,0,0,50,12,0,12,0,0,0,224,44,152,0,4,0
2,Camden 001C,Highgate,0,254.000,0,20,4,340,20,0,48,0,92.000,184.000,0,0,0,8,0,0,16,0,0,0,444.000,0,0,68,8,16,16,0,0,0,0,0,0.000,184,32,152,48,4,72.000,0,40,226.000,0,20,20,0,8,0,8,56,0,4,0,0,0,0,0,176,24,24,284,4,0,32,0,0,8,0,0,0,0,8,0,0,0,34,4,0,0,0,4,0,124,28,100,0,0,0
3,Camden 001D,Highgate,8,734.000,0,16,4,116,132,0,52,4,180.000,584.000,0,0,0,12,0,4,8,0,0,0,894.000,0,0,46,4,4,12,4,0,8,0,0,3696.000,220,0,114,36,8,56.000,0,28,13200.000,4,72,24,0,0,116,4,56,4,0,0,0,4,0,12,84,20,0,160,0,0,36,0,0,12,0,32,0,0,16,0,0,0,12,0,0,0,0,4,16,92,48,120,0,0,0
4,Camden 001E,Highgate,0,128.000,0,0,0,284,8,4,104,4,4.000,88.000,0,0,0,4,0,4,4,0,8,0,198.000,0,4,152,16,4,44,0,0,12,0,0,0.000,132,12,68,16,12,88.000,4,12,242.000,28,44,28,4,0,0,0,12,16,4,4,20,0,0,0,88,20,8,296,4,0,56,0,0,0,0,0,0,0,16,0,0,4,14,4,0,4,0,0,0,200,16,44,0,0,0


In [33]:
# Create a subframe for testing
ethnicity = ethnicityDf.copy()

# Specify the religion columns to consider
ethnicity_columns = [
    "Afghan",
    "African",
    "African/Arab",
    "Albanian",
    "Anglo Indian",
    "Any other ethnic group",
    "Arab",
    "Argentinian",
    "Australian/New Zealander",
    "Baltic States",
    "Bangladeshi, British Bangladeshi",
    "Black British",
    "Black European",
    "Black and Asian",
    "Black and Chinese",
    "Black and White",
    "Black/African American",
    "Bosnian",
    "Brazilian",
    "British Asian",
    "Burmese",
    "Cambodia",
    "Caribbean",
    "Caribbean Asian",
    "Chilean",
    "Chinese",
    "Chinese and White",
    "Colombian",
    "Commonwealth of (Russian) Independent States",
    "Croatian",
    "Cuban",
    "Cypriot (part not stated)",
    "East African Asian",
    "Ecuadorian",
    "English/Welsh/Scottish/Northern Irish/British",
    "European Mixed",
    "Filipino",
    "Greek",
    "Greek Cypriot",
    "Gypsy or Irish Traveller",
    "Indian or British Indian",
    "Indonesian",
    "Iranian",
    "Irish",
    "Israeli",
    "Italian",
    "Japanese",
    "Kashmiri",
    "Korean",
    "Kosovan",
    "Kurdish",
    "Latin/South/Central American",
    "Malaysian",
    "Mexican",
    "Moroccan",
    "Multi-ethnic islands",
    "Nepalese (includes Gurkha)",
    "Nigerian",
    "North African",
    "North American",
    "Other Eastern European",
    "Other Middle East",
    "Other Western European",
    "Pakistani or British Pakistani",
    "Peruvian",
    "Polish",
    "Polynesia/Micronesia/Melanesia",
    "Punjabi",
    "Serbian",
    "Sinhalese",
    "Somali",
    "Somalilander",
    "South Asian and Chinese",
    "Sri Lankan",
    "Taiwanese",
    "Tamil",
    "Thai",
    "Turkish",
    "Turkish Cypriot",
    "Venezuelan",
    "Vietnamese",
    "White African",
    "White Caribbean",
    "White and Arab",
    "White and Asian",
    "White and Black African",
    "White and Black Caribbean",
    "White and East Asian",
    "White and North African or Middle Eastern",
    "White and South Asian",
]
ethnicityDf.shape

(4837, 92)

In [34]:
# Function to return n-th largest categories
def nth_largest_categories(row, n):
    sorted_row = row.sort_values(ascending=False)
    if n <= len(sorted_row):
        return sorted_row.index[n - 1]
    else:
        return np.nan


# Apply function to find the majority first, second, and third most common Ethnicity
ethnicity["Major Ethnicity"] = ethnicity[ethnicity_columns].apply(
    lambda row: nth_largest_categories(row, 1), axis=1
)
ethnicity["Second Major Ethnicity"] = ethnicity[ethnicity_columns].apply(
    lambda row: nth_largest_categories(row, 2), axis=1
)
ethnicity["Third Major Ethnicity"] = ethnicity[ethnicity_columns].apply(
    lambda row: nth_largest_categories(row, 3), axis=1
)
# Display the first few rows to verify the 'Majority Ethnicities' column
ethnicity[
    ["LSOA Name", "Major Ethnicity", "Second Major Ethnicity", "Third Major Ethnicity"]
]


# Perform one-hot encoding for 'Majority Ethnicity', 'Second Ethnicity', 'Third Ethnicity'
majority_onehot = pd.get_dummies(ethnicity["Major Ethnicity"], prefix="Majority")
second_onehot = pd.get_dummies(ethnicity["Second Major Ethnicity"], prefix="Second")
third_onehot = pd.get_dummies(ethnicity["Third Major Ethnicity"], prefix="Third")

# Concatenate the original DataFrame with the new one-hot encoded DataFrames
ethnicity_encoded = pd.concat(
    [ethnicity, majority_onehot, second_onehot, third_onehot], axis=1
)
# Drop the original categorical columns after encoding (attempting clustering without one-hot encoding can be an option to try)
ethnicity_encoded.drop(
    [
        "Afghan",
        "African",
        "African/Arab",
        "Albanian",
        "Anglo Indian",
        "Any other ethnic group",
        "Arab",
        "Argentinian",
        "Australian/New Zealander",
        "Baltic States",
        "Bangladeshi, British Bangladeshi",
        "Black British",
        "Black European",
        "Black and Asian",
        "Black and Chinese",
        "Black and White",
        "Black/African American",
        "Bosnian",
        "Brazilian",
        "British Asian",
        "Burmese",
        "Cambodia",
        "Caribbean",
        "Caribbean Asian",
        "Chilean",
        "Chinese",
        "Chinese and White",
        "Colombian",
        "Commonwealth of (Russian) Independent States",
        "Croatian",
        "Cuban",
        "Cypriot (part not stated)",
        "East African Asian",
        "Ecuadorian",
        "English/Welsh/Scottish/Northern Irish/British",
        "European Mixed",
        "Filipino",
        "Greek",
        "Greek Cypriot",
        "Gypsy or Irish Traveller",
        "Indian or British Indian",
        "Indonesian",
        "Iranian",
        "Irish",
        "Israeli",
        "Italian",
        "Japanese",
        "Kashmiri",
        "Korean",
        "Kosovan",
        "Kurdish",
        "Latin/South/Central American",
        "Malaysian",
        "Mexican",
        "Moroccan",
        "Multi-ethnic islands",
        "Nepalese (includes Gurkha)",
        "Nigerian",
        "North African",
        "North American",
        "Other Eastern European",
        "Other Middle East",
        "Other Western European",
        "Pakistani or British Pakistani",
        "Peruvian",
        "Polish",
        "Polynesia/Micronesia/Melanesia",
        "Punjabi",
        "Serbian",
        "Sinhalese",
        "Somali",
        "Somalilander",
        "South Asian and Chinese",
        "Sri Lankan",
        "Taiwanese",
        "Tamil",
        "Thai",
        "Turkish",
        "Turkish Cypriot",
        "Venezuelan",
        "Vietnamese",
        "White African",
        "White Caribbean",
        "White and Arab",
        "White and Asian",
        "White and Black African",
        "White and Black Caribbean",
        "White and East Asian",
        "White and North African or Middle Eastern",
        "White and South Asian",
        "Major Ethnicity",
        "Second Major Ethnicity",
        "Third Major Ethnicity",
    ],
    axis=1,
    inplace=True,
)

# Rename 'Ward' column to 'Wards'
ethnicity_encoded.rename(columns={"Ward": "Wards"}, inplace=True)
# Display the first few rows to verify the changes
ethnicity_encoded.head()
ethnicity_encoded.shape

(4837, 66)

In [35]:
# Merge merged_df with ethnicity_encoded to combine related data
# 'Lower layer super output area' in merged_df and 'LSOA Name' in ethnicity_encoded are needed for accurate merging
merged_df = merged_df.merge(
    ethnicity_encoded,
    left_on="Lower layer super output area",
    right_on="LSOA Name",
    how="left",
)

# Drop 'LSOA Name' and 'Wards' from the merged DataFrame
merged_df.drop(["LSOA Name", "Wards"], axis=1, inplace=True, errors="ignore")

# Display the first few rows to verify the changes
merged_df.head()
#merged_df.shape
#merged_df.isnull().sum()

,District,Lower layer super output area,Ward,Postcode,Postcode district,London zone,Latitude,Longitude,Average Income,Total Crimes,Index of Multiple Deprivation,Rural/urban,Average Rent Price,Budget-Friendly Rent Price,Median,Premium Rents Price,Mean average price,Annual mean average flat price,Annual mean average terraced price,Annual mean average semi price,Annual mean average detached price,"Agriculture, Forestry & Fishing",Manufacturing & Production,Building & Construction,Automotive Services,Grocery stores & Wholesale,Shops & Retail,Transportation & Storage,Hotels & Restaurants,Tech & Communication,Finance & Insurance,Real Estate Services,"Professional, Scientific & Technical",Business Administration & Support Services,Public Sector & Defence,Education,Healthcar Services,"Arts, Entertainment, Recreation & Other Services",Total parks in District,Majority_Christian,Majority_Hindu,Majority_Jewish,Majority_Muslim (Islam),Majority_No religion,Majority_Religion not stated,Majority_Sikh,Second_Christian,Second_Hindu,Second_Jewish,Second_Muslim (Islam),Second_No religion,Second_Religion not stated,Second_Sikh,Third_Christian,Third_Hindu,Third_Jewish,Third_Muslim (Islam),Third_No religion,Third_Other religion,Third_Religion not stated,Third_Sikh,Majority_African,Majority_Any other ethnic group,Majority_Arab,Majority_Australian/New Zealander,"Majority_Bangladeshi, British Bangladeshi",Majority_Caribbean,Majority_Chinese,Majority_Greek,Majority_Indian or British Indian,Majority_Irish,Majority_Korean,Majority_Other Western European,Majority_Pakistani or British Pakistani,Majority_Sri Lankan,Majority_Turkish,Majority_White and Asian,Second_Afghan,Second_African,Second_Any other ethnic group,Second_Arab,Second_Australian/New Zealander,"Second_Bangladeshi, British Bangladeshi",Second_Black British,Second_Caribbean,Second_Chinese,Second_English/Welsh/Scottish/Northern Irish/British,Second_European Mixed,Second_Filipino,Second_Greek,Second_Greek Cypriot,Second_Indian or British Indian,Second_Irish,Second_Japanese,Second_Korean,Second_North American,Second_Other Western European,Second_Pakistani or British Pakistani,Second_Sri Lankan,Second_Turkish,Second_White and Black Caribbean,Third_African,Third_Any other ethnic group,Third_Arab,Third_Australian/New Zealander,"Third_Bangladeshi, British Bangladeshi",Third_Black British,Third_Caribbean,Third_Chinese,Third_English/Welsh/Scottish/Northern Irish/British,Third_European Mixed,Third_Greek,Third_Greek Cypriot,Third_Indian or British Indian,Third_Irish,Third_Japanese,Third_Korean,Third_North American,Third_Other Western European,Third_Pakistani or British Pakistani,Third_Polish,Third_Sri Lankan,Third_Turkish,Third_White and Asian,Third_White and Black Caribbean
0,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5PU,RM6,5,51.588,0.135,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Barking and Dagenham,Barking and Dagenham 001A,Chadwell Heath,RM6 5LJ,RM6,5,51.584,0.137,43600,334,5097,Urban major conurbation,1226.333,1095.833,1202.833,1376.667,355457.750,359436.750,235446.500,393380.000,444454.250,15,325,"1,965",270,385,775,760,380,465,60,175,725,770,0,130,340,345,73,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,Tru

### Exporting the Final Dataset

In [ ]:
# File path where the CSV file will be saved, including the filename and extension
output_file_path = (
    "/Users/alessandrosuarez_/Desktop/cleanedDatasets/clean_final_data.csv"
)
# Save the DataFrame to CSV
#merged_df.to_csv(output_file_path, index=False)
print(f"DataFrame saved to {output_file_path}")